## Creating csv file with metainformation for pinterest df:

In [2]:
import os
import csv

data_dir = "C:/Deep Learning Project/data/pinterest" 
csv_file = "C:/Deep Learning Project/data/pinterest/metadata.csv"

# Създаване на CSV файл
with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["image_path", "room_type", "style"])
    
    # Минава през всички поддиректории и файлове
    for room_type in os.listdir(data_dir):
        room_path = os.path.join(data_dir, room_type)
        if os.path.isdir(room_path):
            for style in os.listdir(room_path):
                style_path = os.path.join(room_path, style)
                if os.path.isdir(style_path):
                    for image in os.listdir(style_path):
                        if image.endswith((".jpg", ".jpeg", ".png")):  # Само изображения
                            image_path = os.path.join(room_path, style, image)
                            writer.writerow([image_path, room_type, style])

print(f"CSV файлът е създаден: {csv_file}")


CSV файлът е създаден: ../data/metadata.csv


In [3]:
import pandas as pd

df = pd.read_csv("C:/Deep Learning Project/data/pinterest/metadata.csv")

df["image_path"] = df["image_path"].str.replace("\\", "/")

In [4]:
df

,image_path,room_type,style
0,../data/raw/bathroom/boho/bathroom_boho_0.jpg,bathroom,boho
1,../data/raw/bathroom/boho/bathroom_boho_1.jpg,bathroom,boho
2,../data/raw/bathroom/boho/bathroom_boho_10.jpg,bathroom,boho
3,../data/raw/bathroom/boho/bathroom_boho_100.jpg,bathroom,boho
4,../data/raw/bathroom/boho/bathroom_boho_101.jpg,bathroom,boho
...,...,...,...
4134,../data/raw/living_room/scandinavian/living_ro...,living_room,scandinavian
4135,../data/raw/living_room/scandinavian/living_ro...,living_room,scandinavian
4136,../data/raw/living_room/scandinavian/living_ro...,living_room,scandinavian
4137,../data/raw/living_room/scandinavian/living_ro...,living_room,scandinavian


In [5]:
import os

total_images = sum([len(files) for _, _, files in os.walk("../data/raw")])
print(f"Total images in folders: {total_images}")
print(f"Total images in CSV file: {len(df)}")

Total images in folders: 4142
Total images in CSV file: 4139


In [6]:
df.groupby(["room_type", "style"]).size()

room_type    style       
bathroom     boho            122
             industrial      155
             minimalist      168
             modern          138
             scandinavian    121
bedroom      boho            140
             industrial      203
             minimalist      184
             modern          201
             scandinavian    153
kitchen      boho            472
             industrial      300
             minimalist      189
             modern          387
             scandinavian    150
living_room  boho            184
             industrial      201
             minimalist      203
             modern          300
             scandinavian    168
dtype: int64

In [7]:
from sklearn.preprocessing import LabelEncoder

# Encode labels
encoder = LabelEncoder()
df['encoded_label'] = encoder.fit_transform(df['room_type'] + '_' + df['style'])


In [13]:
df.head()

,image_path,room_type,style,encoded_label
0,../data/raw/bathroom/boho/bathroom_boho_0.jpg,bathroom,boho,0
1,../data/raw/bathroom/boho/bathroom_boho_1.jpg,bathroom,boho,0
2,../data/raw/bathroom/boho/bathroom_boho_10.jpg,bathroom,boho,0
3,../data/raw/bathroom/boho/bathroom_boho_100.jpg,bathroom,boho,0
4,../data/raw/bathroom/boho/bathroom_boho_101.jpg,bathroom,boho,0


In [14]:
from sklearn.model_selection import train_test_split

# Train-test split
train_df, test_df = train_test_split(
    df, 
    test_size=0.2, 
    stratify=df['encoded_label'], 
    random_state=42
)

# Train-validation split
train_df, val_df = train_test_split(
    train_df, 
    test_size=0.2, 
    stratify=train_df['encoded_label'], 
    random_state=42
)

In [19]:
print("Train distribution:")
print(train_df['encoded_label'].value_counts(normalize=True))

print("Validation distribution:")
print(val_df['encoded_label'].value_counts(normalize=True))

print("Test distribution:")
print(test_df['encoded_label'].value_counts(normalize=True))


Train distribution:
encoded_label
10    0.114048
13    0.093656
11    0.072508
18    0.072508
17    0.048716
16    0.048716
8     0.048716
6     0.048716
12    0.045695
15    0.044562
7     0.044184
19    0.040408
2     0.040408
1     0.037387
9     0.037009
14    0.036254
5     0.033988
3     0.033610
4     0.029456
0     0.029456
Name: proportion, dtype: float64
Validation distribution:
encoded_label
10    0.114630
13    0.093514
18    0.072398
11    0.072398
6     0.049774
17    0.049774
16    0.048265
8     0.048265
7     0.045249
12    0.045249
15    0.043741
2     0.040724
19    0.040724
1     0.037707
14    0.036199
9     0.036199
5     0.033183
3     0.033183
0     0.030166
4     0.028658
Name: proportion, dtype: float64
Test distribution:
encoded_label
10    0.113527
13    0.092995
11    0.072464
18    0.072464
17    0.049517
6     0.049517
8     0.048309
16    0.048309
12    0.045894
7     0.044686
15    0.044686
2     0.041063
19    0.041063
9     0.037440
1     0.037440
14 

In [21]:
df.encoded_label.value_counts()

encoded_label
10    472
13    387
11    300
18    300
17    203
6     203
16    201
8     201
12    189
15    184
7     184
19    168
2     168
1     155
9     153
14    150
5     140
3     138
0     122
4     121
Name: count, dtype: int64

In [20]:
# Verify counts
print("Train counts:")
print(train_df['encoded_label'].value_counts())

print("Validation counts:")
print(val_df['encoded_label'].value_counts())

print("Test counts:")
print(test_df['encoded_label'].value_counts())


Train counts:
encoded_label
10    302
13    248
11    192
18    192
17    129
16    129
8     129
6     129
12    121
15    118
7     117
19    107
2     107
1      99
9      98
14     96
5      90
3      89
4      78
0      78
Name: count, dtype: int64
Validation counts:
encoded_label
10    76
13    62
18    48
11    48
6     33
17    33
16    32
8     32
7     30
12    30
15    29
2     27
19    27
1     25
14    24
9     24
5     22
3     22
0     20
4     19
Name: count, dtype: int64
Test counts:
encoded_label
10    94
13    77
11    60
18    60
17    41
6     41
8     40
16    40
12    38
7     37
15    37
2     34
19    34
9     31
1     31
14    30
5     28
3     27
4     24
0     24
Name: count, dtype: int64


In [18]:
train_df.to_csv("../data/train_data.csv", index=False)
val_df.to_csv("../data/val_data.csv", index=False)
test_df.to_csv("../data/test_data.csv", index=False)


In [ ]:
import os
import csv
from colorthief import ColorThief

# Функция за извличане на основния цвят на изображението
def get_color_palette(image_path):
    color_thief = ColorThief(image_path)
    # Вземаме основния цвят и генерираме палитра
    dominant_color = color_thief.get_color(quality=1)
    palette = color_thief.get_palette(color_count=6)
    return dominant_color, palette

In [ ]:
get_color_palette("datasets/baby_and_kids_images/image_0.jpg")